In [ ]:
1
import numpy as np

def print_latex_matrix(matrix, b=None):
    """Print matrix in LaTeX format"""
    latex = "\\begin{bmatrix}\n"
    for i in range(len(matrix)):
        row = " & ".join([f"{x:.2f}" for x in matrix[i]])
        if b is not None:
            row += f" & | & {b[i]:.2f}"
        latex += row + " \\\\\n"
    latex += "\\end{bmatrix}"
    print(latex)

def print_latex_equation(matrix, b):
    """Print system of equations in LaTeX format"""
    vars = ['x_1', 'x_2', 'x_3']
    equations = []
    for i in range(len(matrix)):
        terms = []
        for j in range(len(matrix[i])):
            coef = matrix[i][j]
            if coef != 0:
                if coef == 1:
                    terms.append(f"{vars[j]}")
                elif coef == -1:
                    terms.append(f"-{vars[j]}")
                else:
                    terms.append(f"{coef:.2f}{vars[j]}")
        equation = " + ".join(terms).replace("+ -", "- ")
        equations.append(f"{equation} = {b[i]:.2f}")
    print("\\begin{align*}")
    print("\\\\\n".join(equations))
    print("\\end{align*}")

# Initial system of equations
A = np.array([
    [1, -1, 3],
    [1, 1, 0],
    [3, -2, 1]
], dtype=float)

b = np.array([2, 4, 1], dtype=float)

print("Original system:")
print_latex_equation(A, b)
print("\nOriginal augmented matrix:")
print_latex_matrix(A, b)

# Gaussian Elimination
# Step 1: Eliminate x₁ from R2 using R1
A[1] = A[1] - A[0]  # R2 = R2 - R1
b[1] = b[1] - b[0]

# Step 2: Eliminate x₁ from R3 using R1
A[2] = A[2] - 3*A[0]  # R3 = R3 - 3R1
b[2] = b[2] - 3*b[0]

print("\nAfter first elimination (x₁):")
print_latex_matrix(A, b)

# Step 3: Eliminate x₂ from R3 using R2
A[2] = A[2] - 0.5*A[1]  # R3 = R3 - (1/2)R2
b[2] = b[2] - 0.5*b[1]

print("\nFinal upper triangular matrix:")
print_latex_matrix(A, b)

# Backward substitution
x = np.zeros(3)

# Solve for x3
x[2] = b[2]/A[2,2]  # -4/-6.5

# Solve for x2
x[1] = (b[1] - A[1,2]*x[2])/A[1,1]  # (2 - (-3)*x3)/2

# Solve for x1
x[0] = (b[0] - A[0,1]*x[1] - A[0,2]*x[2])/A[0,0]  # 2 - (-1)*x2 - 3*x3

print("\nSolution:")
for i in range(3):
    print(f"x_{i+1} = {x[i]:.2f}")

In [ ]:
2

In [ ]:
3. 3 If we are not given T matrix it is impossible, as finding the inverse of a matrix is O(n^3).
Assuming we are given T, L, U, and if b:

    Step 1: Calculate c = Tb which takes O(n^2) FLOPS

    c = vector_create(n)

    for i = 0 to n-1:
        sum = 0
        for j = 0 to n-1:
            // Multiply each element of T's row with the corresponding element of b
            sum = sum + T[i][j] * b[j]
        // Store the sum in the corresponding element of c
        c[i] = sum

    LUx=c
    Step 2:Set Ux=y and Ly=c
    Step 3: Solving Ly=c takes forward sub
        for i = 0 to n-1:
            y[i] = c[i]
            for j = 0 to i-1:
                y[i] = y[i] - L[i][j] * y[j]

    Step 4:Solving Ux=y takes backward sub
        for i = n-1 downto 0:
            x[i] = y[i]
            for j = i+1 to n-1:
                x[i] = x[i] - U[i][j] * x[j]
            x[i] = x[i] / U[i][i]

    Given that backwards and forwards sub are O(n^2), Step 1, Step 3, and Step 4 together take O(n^2) and Step 2 takes O(1)

Therefore This algorithm takes O(n^2)


In [ ]:
5
import numpy as np

def print_system(A, b):
    """Print the initial system in LaTeX format"""
    print("Original system A'x = b':")
    print("\\begin{equation*}")
    print("A' = \\begin{bmatrix}")
    for row in A:
        print(" & ".join(map(str, row)) + " \\\\")
    print("\\end{bmatrix}, \\quad")
    print("b' = \\begin{bmatrix}")
    print(" \\\\\n".join(map(str, b)))
    print("\\end{bmatrix}")
    print("\\end{equation*}\n")

def back_substitution(A, b):
    """Perform back substitution and print steps in LaTeX"""
    n = len(b)
    x = np.zeros(n)

    print("Steps:")
    print("\\begin{align*}")

    # Start from the last equation
    x[n-1] = b[n-1] / A[n-1][n-1]
    print(f"x_{n} &= \\frac{{{b[n-1]}}}{{{A[n-1][n-1]}}} = {x[n-1]:.0f}")

    # Work backwards through the equations
    for i in range(n-2, -1, -1):
        sum_term = sum(A[i][j] * x[j] for j in range(i+1, n))

        # Print the equation
        equation_terms = []
        for j in range(i, n):
            coef = A[i][j]
            if coef != 0:
                if j == i:
                    equation_terms.append(f"{coef}x_{j+1}")
                else:
                    equation_terms.append(f"{coef}({x[j]:.0f})")
        equation = " + ".join(equation_terms).replace("+ -", "- ")
        print(f"& \\text{{From row {i+1}:}} \\\\")
        print(f"{equation} &= {b[i]} \\\\")

        # Calculate and print the solution for this variable
        x[i] = (b[i] - sum_term) / A[i][i]
        print(f"x_{i+1} &= {x[i]:.0f} \\\\")

    print("\\end{align*}\n")
    return x

def print_solution(x):
    """Print the solution vector in LaTeX format"""
    print("Solution:")
    print("\\begin{equation*}")
    print("x = \\begin{bmatrix}")
    print(" \\\\\n".join(f"{val:.0f}" for val in x))
    print("\\end{bmatrix}")
    print("\\end{equation*}\n")

def verify_solution(A, b, x):
    """Verify the solution and print in LaTeX format"""
    print("Verification:")
    print("\\begin{align*}")
    for i in range(len(b)):
        terms = []
        for j in range(len(x)):
            if A[i][j] != 0:
                terms.append(f"{A[i][j]}({x[j]:.0f})")
        equation = " + ".join(terms).replace("+ -", "- ")
        result = sum(A[i][j] * x[j] for j in range(len(x)))
        print(f"\\text{{Row {i+1}:}} {equation} &= {result:.0f} = {b[i]} \\checkmark \\\\")
    print("\\end{align*}")

def main():
    # Define the system
    A = np.array([
        [5, 6, 7, 8],
        [0, 4, 3, 2],
        [0, 0, -1, -2],
        [0, 0, 0, 1]
    ])

    b = np.array([26, 9, -3, 1])

    # Solve and display the solution
    print_system(A, b)
    x = back_substitution(A, b)
    print_solution(x)
    verify_solution(A, b, x)

if __name__ == "__main__":
    main()

In [ ]:
6 this article states that https://mathed.miamioh.edu/index.php/ggbj/article/view/177/144 states that "Theorem 1(Existence).Suppose thatA∈Mnis rankk.  Ifdet(Aj)6= 0for allj= 1,...k, (allleading principal minors are non-zero), thenAhas a LU factorization. Moreover, ifk=n, then thisfactorization is unique."

This is proved by https://math.stackexchange.com/questions/2951461/lu-factorization-of-a-nonsingular-matrix-exists-if-and-only-if-all-leading-princ


Therefore if A=B-B^T => B_kk for all k = 1 to n do not swap when B^T => A has a diaganol of zeros=>the first column of eliminations is not possible => pivot must be carried out before the first step of Gauss Jordan can be performed